In [1]:
import cv2 
import os 
import numpy as np 
from random import shuffle
from tqdm import tqdm
import matplotlib.pyplot as plt 
import tflearn 
from tflearn.layers.conv import conv_2d, max_pool_2d 
from tflearn.layers.core import input_data, dropout, fully_connected 
from tflearn.layers.estimator import regression 
import tensorflow as tf
from numpy import expand_dims
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.preprocessing.image import ImageDataGenerator

c:\users\admin\appdata\local\programs\python\python37\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
c:\users\admin\appdata\local\programs\python\python37\lib\site-packages\tensorflow\python\framework\dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
c:\users\admin\appdata\local\programs\python\python37\lib\site-packages\tensorflow\python\framework\dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
c:\users\admin\appd

curses is not supported on this machine (please install/reinstall curses for an optimal experience)
Instructions for updating:
Colocations handled automatically by placer.


Using TensorFlow backend.


In [2]:
TRAIN_DIR = 'C:\\Users\\Admin\\Desktop\\images dataset\\TRAIN'
TEST_DIR = 'C:\\Users\\Admin\\Desktop\\images dataset\\TEST'
IMG_SIZE = 50
LR = 1e-3

MODEL_NAME = 'Garbage_Segregator-{}-{}.model'.format(LR, '6conv-basic') 

In [3]:
def label_img(img, word_label): 
    if word_label == 'metal':
            return [1, 0, 0, 0, 0] 
    elif word_label == 'paper':
            return [0, 1, 0, 0, 0]
    elif word_label == 'cardboard':
            return [0, 0, 1, 0, 0]
    elif word_label == 'glass':
            return [0, 0, 0, 1, 0]
    elif word_label == 'plastic':
            return [0, 0, 0, 0, 1]

In [4]:
def create_train_data():
    training_data = [] 
    for word_label in ['metal', 'paper', 'cardboard', 'glass', 'plastic']:
            for img in tqdm(os.listdir(os.path.join(TRAIN_DIR, word_label))):
                label = label_img(img, word_label)
                path = os.path.join(TRAIN_DIR, word_label, img)
                img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
                img = cv2.resize(img, (IMG_SIZE, IMG_SIZE)) 
                training_data.append([np.array(img), np.array(label)]) 
                img = load_img(path)
                data = img_to_array(img)
                samples = expand_dims(data, 0)
                datagen = ImageDataGenerator(horizontal_flip=True)
                it = datagen.flow(samples, batch_size=1)
                for i in range(9):
                    batch = it.next()
                    image = batch[0].astype('uint8')
                    imgd = tf.image.encode_jpeg(image)
                    #training_data.append([np.array(imgd), np.array(label)]
    shuffle(training_data)
    np.save('train_data.npy', training_data)
    return training_data 

In [5]:
def process_test_data(): 
    
    testing_data = []
    for word_label in ['metal', 'paper', 'cardboard', 'glass', 'plastic']:
            for img in tqdm(os.listdir(os.path.join(TEST_DIR, word_label))): 
                path = os.path.join(TEST_DIR, word_label, img) 
                img_num = label_img(img, word_label) 
                img = cv2.imread(path, cv2.IMREAD_GRAYSCALE) 
                img = cv2.resize(img, (IMG_SIZE, IMG_SIZE)) 
                testing_data.append([np.array(img), img_num]) 
    shuffle(testing_data) 
    np.save('test_data.npy', testing_data) 
    return testing_data 

In [6]:
train_data = create_train_data() 
test_data = process_test_data() 

100%|██████████████████████████████████████████████████████████████████████████████████| 82/82 [00:18<00:00,  4.54it/s]


In [7]:
tf.reset_default_graph() 
convnet = input_data(shape =[None, IMG_SIZE, IMG_SIZE, 1], name ='input') 

convnet = conv_2d(convnet, 32, 5, activation ='relu') 
convnet = max_pool_2d(convnet, 5) 

convnet = conv_2d(convnet, 64, 5, activation ='relu') 
convnet = max_pool_2d(convnet, 5) 

convnet = conv_2d(convnet, 128, 5, activation ='relu') 
convnet = max_pool_2d(convnet, 5)

convnet = conv_2d(convnet, 64, 5, activation ='relu') 
convnet = max_pool_2d(convnet, 5) 

convnet = conv_2d(convnet, 32, 5, activation ='relu') 
convnet = max_pool_2d(convnet, 5) 

convnet = fully_connected(convnet, 1024, activation ='relu') 
convnet = dropout(convnet, 0.8) 

convnet = fully_connected(convnet, 5, activation ='softmax') 
convnet = regression(convnet, optimizer ='adam', learning_rate = LR, loss ='categorical_crossentropy', name ='targets') 

model = tflearn.DNN(convnet, tensorboard_dir ='log') 

Instructions for updating:
Use tf.initializers.variance_scaling instead with distribution=uniform to get equivalent behavior.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
Use tf.cast instead.


In [8]:
train = train_data
test = test_data 

In [9]:
X = np.array([i[0] for i in train]).reshape(-1, IMG_SIZE, IMG_SIZE, 1) 
Y = [i[1] for i in train] 
test_x = np.array([i[0] for i in test]).reshape(-1, IMG_SIZE, IMG_SIZE, 1) 
test_y = [i[1] for i in test] 

In [10]:
model.fit({'input': X}, {'targets': Y}, n_epoch = 25,validation_set =({'input': test_x}, {'targets': test_y}), 
           show_metric = True, run_id = MODEL_NAME) 
model.save(MODEL_NAME) 

Training Step: 849  | total loss: 0.75077 | time: 5.875s
| Adam | epoch: 025 | loss: 0.75077 - acc: 0.7520 -- iter: 2112/2146
Training Step: 850  | total loss: 0.72062 | time: 7.059s
| Adam | epoch: 025 | loss: 0.72062 - acc: 0.7565 | val_loss: 0.43392 - val_acc: 0.8363 -- iter: 2146/2146
--
INFO:tensorflow:C:\Users\Admin\Desktop\TARP\Garbage_Segregator-0.001-6conv-basic.model is not in all_model_checkpoint_paths. Manually adding it.
